# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [ ]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [ ]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=10_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

#### Análise de Target

In [ ]:
df['loan_status'].value_counts(dropna=False, normalize=True) * 100

In [ ]:
TARGET = 'target_risco_credito'

status_de_risco = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    #"Late (16-30 days)"
]
df[TARGET] = df["loan_status"].isin(status_de_risco).astype(int)

df[TARGET].value_counts(dropna=False, normalize=True) * 100

### Safras

In [ ]:
#date_col=['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d','next_pymnt_d']

temporal_columns = [
    "issue_d",                      # Data de emissão do empréstimo
    "earliest_cr_line",             # Primeira linha de crédito do cliente
    "last_pymnt_d",                 # Último pagamento realizado
    "last_credit_pull_d",           # Última consulta ao crédito
    "next_pymnt_d",                 # Próximo pagamento previsto (se aplicável)
    #"last_pymnt_amnt",             # (associada à data de pagamento)
    "debt_settlement_flag_date",    # Data em que houve acordo de dívida
    "settlement_date"               # Data em que o acordo foi fechado
]


# tratamento para colunas de data
for col in temporal_columns:
    df[col] = pd.to_datetime(df[col], format="%b-%Y")


df["safra"] = df["issue_d"].dt.to_period("M")#.astype(str)  # formato 'YYYY-MM'

print(df["safra"].min())
print(df["safra"].max())

### Captura Inicial

In [ ]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    target_col=TARGET,
    default_keys=["id"]
)

trail.take_snapshot(df, name="amostra")

In [ ]:
trail.describe_snapshot("amostra")

#### Limpeza com Vassoura

In [ ]:
assert set(df[TARGET].dropna().unique()) == {0, 1}, "TARGET não é binário!"

In [9]:
vs = Vassoura(
    df,
    id_cols = ['id','member_id'],
    date_cols=['safra'],
    ignore_cols=['url']+temporal_columns,
    drop_ignored=True,
    target_col=TARGET,
    verbose='basic',
    
    engine='polars',
    
    adaptive_sampling=True,

    heuristics=[
        'missing',
        'variance',
        "iv",
        "graph_cut",
        "corr",
        "vif",
        'ks_separation',
        #"psi_stability",
        #"perm_importance_lgbm",
        #"partial_corr_cluster",
        #"drift_vs_target_leakage",
    ],
    
    thresholds={
        "missing": 0.60,
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,
    },
    n_steps=2,         # controla quantas remoções por iteração
    vif_n_steps=2      # VIF será sempre 1-step (a menos que você altere)
)

df_limpo = vs.run(recompute=True)

INFO | Tipos detectados: 44 numéricas, 10 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 44 variáveis numéricas (engine=pandas)


  → dropped ['bc_util'] (corr>0.85)
  → dropped ['mths_since_rcnt_il'] (corr>0.85)


INFO | Tipos detectados: 42 numéricas, 10 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 42 variáveis numéricas (engine=pandas)


[Vassoura] VIF heuristic (thr=5) vif_n_steps=2


INFO | Tipos detectados: 42 numéricas, 10 categóricas
INFO | Desconsiderando 2211 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 42 variáveis


...

  → dropped ['annual_inc'] (vif>5)


INFO | Tipos detectados: 20 numéricas, 10 categóricas
INFO | Desconsiderando 1170 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 20 variáveis


  → dropped ['num_rev_tl_bal_gt_0'] (vif>5)


INFO | Tipos detectados: 19 numéricas, 10 categóricas
INFO | Desconsiderando 1170 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 19 variáveis
INFO | Tipos detectados: 17 numéricas, 8 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 17 variáveis numéricas (engine=pandas)


  → dropped ['emp_title', 'emp_length', 'zip_code', 'addr_state', 'revol_bal', 'total_bal_il'] (ks<0.05)


In [10]:
df_limpo.head()

,id,member_id,safra,target_risco_credito,term,grade,sub_grade,home_ownership,verification_status,loan_status,purpose,title,dti,inq_last_6mths,out_prncp_inv,collection_recovery_fee,last_pymnt_amnt,open_acc_6m,max_bal_bc,inq_fi,inq_last_12m,avg_cur_bal,bc_open_to_buy,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,percent_bc_gt_75
0,361774,NaN,2015-12,0,36 months,B,B1,MORTGAGE,Not Verified,Fully Paid,debt_consolidation,Debt consolidation,5.10,0.0,0.0,0.0,586.68,1.0,5634.0,0.0,2.0,49598.0,832.0,52.0,3.0,4.0,52.0,3.0,66.7
1,577926,NaN,2015-12,0,36 months,E,E3,RENT,Source Verified,Fully Paid,home_improvement,Home improvement,32.12,2.0,0.0,0.0,5620.34,4.0,2183.0,3.0,5.0,3031.0,7405.0,4.0,4.0,0.0,17.0,4.0,0.0
2,699575,NaN,2015-12,0,36 months,C,C2,MORTGAGE,Not Verified,Fully Paid,debt_consolidation,Debt consolidation,13.12,0.0,0.0,0.0,19906.26,2.0,5826.0,9.0,15.0,11551.0,35864.0,4.0,4.0,1.0,4.0,0.0,0.0
3,811529,NaN,2015-12,0,36 months,A,A4,RENT,Not Verified,Fully Paid,credit_card,Credit card refinancing,7.98,0.0,0.0,0.0,1553.29,2.0,3051.0,0.0,1.0,3085.0,38769.0,4.0,4.0,0.0,4.0,9.0,0.0
4,849795,NaN,2015-12,0,36 months,A,A2,MORTGAGE,Source Verified,Fully Paid,credit_card,Credit card refinancing,8.36,0.0,0.0,0.0,11496.70,0.0,2742.0,2.0,1.0,99056.0,58.0,19.0,19.0,4.0,19.0,6.0,100.0


In [11]:
vs.generate_report("../reports/vassoura_report.html")

INFO | Tipos detectados: 115 numéricas, 27 categóricas
Exception in thread Thread-58 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\JM\AppData\Local\anaconda3\envs\ENV_STONE\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 103: invalid start byte
INFO | Relatório gerado em ..\reports\vassour

'..\\reports\\vassoura_report.html'